In [2]:
import librosa
import webrtcvad
import numpy as np
from pydub import AudioSegment
import os

/opt/anaconda3/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
def detect_voice_segments(flac_path, aggressiveness=3):
    """
    检测FLAC音频中的语音片段和停顿片段
    
    参数:
        flac_path: FLAC音频文件路径
        aggressiveness: 检测 aggressiveness (0-3)，值越高检测越严格
    
    返回:
        包含语音片段和停顿片段的列表，每个元素是(timestamp, duration, is_speech)
    """
    # 加载FLAC文件
    audio, sample_rate = librosa.load(flac_path, sr=16000)  # 转换为16kHz，VAD需要
    
    # 将音频转换为16位整数
    audio = (audio * 32767).astype(np.int16)
    
    # 初始化VAD
    vad = webrtcvad.Vad(aggressiveness)
    
    # 定义帧长（10, 20, 或30毫秒）
    frame_duration_ms = 30
    frame_length = int(sample_rate * frame_duration_ms / 1000)
    frames = [audio[i:i+frame_length] for i in range(0, len(audio), frame_length)]
    
    # 处理最后一帧，确保长度正确
    if len(frames[-1]) < frame_length:
        frames[-1] = np.pad(frames[-1], (0, frame_length - len(frames[-1])), mode='constant')
    
    # 对每个帧进行VAD检测
    is_speech = []
    for frame in frames:
        # 转换为字节
        frame_bytes = frame.tobytes()
        # 检测是否为语音
        is_speech.append(vad.is_speech(frame_bytes, sample_rate))
    
    # 合并连续的语音/非语音片段
    segments = []
    if not is_speech:
        return segments
        
    current_speech = is_speech[0]
    start_time = 0.0
    frame_duration = frame_duration_ms / 1000  # 转换为秒
    
    for i, speech in enumerate(is_speech[1:], 1):
        if speech != current_speech:
            # 结束当前片段
            duration = i * frame_duration - start_time
            segments.append((start_time, duration, current_speech))
            start_time = i * frame_duration
            current_speech = speech
    
    # 添加最后一个片段
    duration = len(is_speech) * frame_duration - start_time
    segments.append((start_time, duration, current_speech))
    
    return segments

In [12]:
def main():
    # 替换为你的FLAC文件路径
    flac_file = "../resources/test/1272/128104/128104_0001.flac"
    
    if not os.path.exists(flac_file):
        print(f"错误: 文件 {flac_file} 不存在")
        return
    
    # 检测语音片段
    segments = detect_voice_segments(flac_file)
    
    # 输出结果
    print("语音活动检测结果:")
    print("时间戳(秒) | 持续时间(秒) | 状态")
    print("-" * 40)
    
    for timestamp, duration, is_speech in segments:
        status = "说话" if is_speech else "停顿"
        print(f"{timestamp:.2f}      | {duration:.2f}        | {status}")

In [14]:
main()

语音活动检测结果:
时间戳(秒) | 持续时间(秒) | 状态
----------------------------------------
0.00      | 0.09        | 说话
0.09      | 0.48        | 停顿
0.57      | 2.49        | 说话
3.06      | 0.03        | 停顿
3.09      | 0.72        | 说话
3.81      | 0.03        | 停顿
3.84      | 1.56        | 说话
5.40      | 0.96        | 停顿
6.36      | 4.65        | 说话
11.01      | 0.12        | 停顿
11.13      | 0.39        | 说话
11.52      | 0.39        | 停顿
11.91      | 0.45        | 说话
12.36      | 0.03        | 停顿
12.39      | 1.83        | 说话
14.22      | 0.99        | 停顿
15.21      | 1.11        | 说话
16.32      | 0.03        | 停顿
16.35      | 1.65        | 说话
18.00      | 0.36        | 停顿
18.36      | 2.85        | 说话
21.21      | 0.90        | 停顿
22.11      | 0.60        | 说话
22.71      | 0.18        | 停顿
22.89      | 2.07        | 说话
24.96      | 0.12        | 停顿
25.08      | 1.86        | 说话
26.94      | 0.09        | 停顿
27.03      | 1.14        | 说话
28.17      | 0.03        | 停顿
28.20      | 1.38        | 说话
29.58 